In [ ]:
from google.colab import drive # Finalized Code ||
drive.mount('/content/drive')

!pip install googletrans==3.1.0a0
import pandas as pd
import numpy as np
import os # Import the os module for file operations
import csv
import re # Import the re module for regular expressions
from googletrans import Translator # Import the Translator class from the googletrans module


# Replace 'Your Folder Path' with the actual path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/Protexxa/nationwide/NationWide_LeakData.csv' # Assign the file path to a variable
df = pd.read_csv(file_path)

print(df.head(20)) # Display the first few rows of the dataframe

garbage_bin = pd.DataFrame()  # Create an empty DataFrame to store outliers

# Create an instance of the Translator class
translator = Translator() # Create an instance of the Translator class

def translate_column_names(df, garbage_bin):
    old_columns = df.columns.tolist() # Get the current column names as a list
    translated_columns = [translator.translate(col, dest= 'en').text for col in old_columns] # Initialize an empty list to store translated column names

    garbage_bin['Old Column Names'] = old_columns
    df.columns = translated_columns # Assign the translated column names to the dataframe's columns

    return df, garbage_bin # Return the updated dataframe

# Call the translate_column_names function with the dataframe and garbage bin
df, garbage_bin = translate_column_names(df, garbage_bin)

# Save the DataFrame with translated column names to a new CSV file
output_file_path = '/content/drive/MyDrive/Protexxa/nationwide/FinalProduct/translated_data.csv'
df.to_csv(output_file_path, index=False)

# Replace 'Your Folder Path' with the actual path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/Protexxa/nationwide/FinalProduct/translated_data.csv' # Assign the file path to a variable
df = pd.read_csv(file_path)


def remove_outliers_zscore(df, threshold=3):
  """
  Removes outliers from a DataFrame using the Z-score method.

  Args:
    df: The DataFrame to process.
    threshold: The Z-score threshold for identifying outliers.

  Returns:
    A tuple containing the DataFrame with outliers removed and a DataFrame
    containing the removed outliers.
  """
  global garbage_bin  # Access the global garbage_bin DataFrame

  for column in df.select_dtypes(include=np.number).columns:  # Iterate only numerical columns
    z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
    outliers = df[z_scores > threshold]

    # Store outliers in the garbage bin with column information
    if not outliers.empty:
      outliers['Outlier_Column'] = column
      garbage_bin = pd.concat([garbage_bin, outliers], ignore_index=True)

    df = df[z_scores <= threshold]  # Remove outliers from the original DataFrame

  return df, garbage_bin

# Example Usage
df, garbage_bin = remove_outliers_zscore(df)

# Now 'df' contains the DataFrame with outliers removed, and 'garbage_bin'
# contains the removed outliers.
print(df.head(20)) # Display the first few rows of the dataframe

# Get the translated column names for 'Frame Number' and 'Engine Number'
frame_number_col = translator.translate('Frame Number', dest='en').text
engine_number_col = translator.translate('Engine Number', dest='en').text

# Save the DataFrame with translated column names to a new CSV file
output_file_path = '/content/drive/MyDrive/Protexxa/nationwide/FinalProduct/removedoutlier_data.csv'
df.to_csv(output_file_path, index=False)

# Create a list of columns to drop
columns_to_drop = ['Unnamed: 21', 'gender', 'Birthday', 'educate', 'monthly salary', 'cell phone', 'address', 'City', 'marriage', 'Province', 'BRAND', 'car model', 'color', 'car series', 'post code', 'industry']

# Iterate through the columns and move them to the garbage bin
for column in columns_to_drop:
    if column in df.columns:
        # Store the entire column in the garbage bin
        garbage_bin[column] = df[column]
        df = df.drop(column, axis=1)
    else:
        print(f"Warning: Column '{column}' not found in DataFrame.")

def handle_invalid_values(df, garbage_bin):
  invalid_values = df.isnull().any(axis=1)  # Identify rows with invalid values

  # Store rows with invalid values in the garbage bin
  garbage_bin = pd.concat([garbage_bin, df[invalid_values]], ignore_index=True)

  df = df.dropna()  # Remove rows with invalid values
  return df, garbage_bin

# Save the modified DataFrame to a new CSV file
df.to_csv('/content/drive/MyDrive/Protexxa/nationwide/FinalProduct/NationWideClean.csv', index=False)

# Save the garbage bin DataFrame to a CSV file
garbage_bin.to_csv('/content/drive/MyDrive/Protexxa/nationwide/Garbage/finalgarbage3.csv', index=False)
